In [ ]:
import SimpleITK as sitk
import os
from sklearn.model_selection import train_test_split

In [ ]:
def save_image(image, output_path):
    # Ensure the image is in the correct format (unsigned char) for PNG
    image = sitk.Cast(sitk.RescaleIntensity(image), sitk.sitkUInt8)
    sitk.WriteImage(image, output_path)

def process_case(case_dir, output_dir, case_name):
    print(f'Processing case: {case_name}...')
    # Paths to the image files
    consensus_path = os.path.join(case_dir, 'Consensus.nii')
    flair_path = os.path.join(case_dir, '3DFLAIR.nii')
    
    # Load the images
    consensus_image = sitk.ReadImage(consensus_path)
    flair_image = sitk.ReadImage(flair_path)

    # Assuming that the third dimension Z is the slice direction
    z_slices = range(consensus_image.GetSize()[2])
    
    # Process each slice and save to the appropriate directory
    for z_slice in z_slices:
        save_image(consensus_image[:, :, z_slice], os.path.join(output_dir, 'labels', f"{case_name}_consensus_{z_slice:03}.png"))
        save_image(flair_image[:, :, z_slice], os.path.join(output_dir, 'images', f"{case_name}_flair_{z_slice:03}.png"))

    print(f'Finished processing case: {case_name}')

# Split and extract slices from MICCAI data set

In [ ]:
# Directories for train and test datasets
output_dir = './data'
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
valid_dir = os.path.join(output_dir, 'valid')

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(valid_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(valid_dir, 'labels'), exist_ok=True)

# Root directory of the cases
data_root_dir = '../0_Data_reg_inter_rigid'

# Get a list of all case directories
case_dirs = [d for d in os.listdir(data_root_dir) if os.path.isdir(os.path.join(data_root_dir, d))]

# First, split the data into a temporary training set and the final testing set
temp_train_cases, test_cases = train_test_split(case_dirs, test_size=3/15, random_state=42)

# Now, split the temporary training set further into the final training set and the validation set
train_cases, valid_cases = train_test_split(temp_train_cases, test_size=2/12, random_state=42)

# Now you have train_cases, valid_cases, and test_cases

# Process each case
for case_name in train_cases:
    case_dir = os.path.join(data_root_dir, case_name)
    process_case(case_dir, train_dir, case_name)

for case_name in test_cases:
    case_dir = os.path.join(data_root_dir, case_name)
    process_case(case_dir, test_dir, case_name)
    
for case_name in valid_cases:
    case_dir = os.path.join(data_root_dir, case_name)
    process_case(case_dir, valid_dir, case_name)

# Convert data for  Yolo (mask)

In [ ]:
import cv2
import numpy as np
from pathlib import Path
import random
import os
from shutil import copy2
# 261,336,336

In [ ]:
# Yolo size is fixed :  we must adapt our data to Yolo first
def resize_and_save_image(image_path, dest_path, scale_dims=(640, 640)):
    """
    Resize an image and save it to the destination path.
    """
    image = cv2.imread(str(image_path))
    image_resized = cv2.resize(image, scale_dims)
    cv2.imwrite(str(dest_path), image_resized)

def find_significant_points(contour, angle_threshold=15):
    """
    Simplify a contour to include points with significant changes in direction.
    """
    def angle_between(p1, p2, p3):
        """
        Calculate the angle between three points.
        """
        vector1 = p2 - p1
        vector2 = p3 - p2
        unit_vector1 = vector1 / np.linalg.norm(vector1)
        unit_vector2 = vector2 / np.linalg.norm(vector2)
        dot_product = np.dot(unit_vector1, unit_vector2)
        angle = np.arccos(dot_product)
        return np.degrees(angle)

    significant_points = [contour[0]]
    for i in range(1, len(contour) - 1):
        angle = angle_between(contour[i - 1][0], contour[i][0], contour[i + 1][0])
        if angle > angle_threshold:
            significant_points.append(contour[i])
    significant_points.append(contour[-1])
    return np.array(significant_points)

def convert_and_save_dataset(src_dir, dest_dir, img_dims=(640, 640)):
    """
    Process datasets, resize images and masks, and convert masks to YOLO format.
    """
    subsets = ['train','val', 'test']
    all_images = []

    for subset in subsets:
        print(f"Processing {subset} subset...")

        images_dir = src_dir / subset / 'images'
        labels_dir = src_dir / subset / 'labels'
        dest_images_dir = dest_dir / subset / 'images'
        dest_labels_dir = dest_dir / subset / 'labels'
        
        dest_images_dir.mkdir(parents=True, exist_ok=True)
        dest_labels_dir.mkdir(parents=True, exist_ok=True)
        
        for image_path in images_dir.glob("*.png"):
            # Resize and save image
            dest_image_path = dest_images_dir / image_path.name
            resize_and_save_image(image_path, dest_image_path, img_dims)

            # Process and convert mask
            mask_name = image_path.name.replace("flair", "consensus")
            mask_path = labels_dir / mask_name
            mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            yolo_annotation_file = dest_labels_dir / mask_name.replace('consensus', 'flair').replace('.png', '.txt')
            with open(yolo_annotation_file, 'w') as file:
                for contour in contours:
                    significant_points = find_significant_points(contour)
                    raveled_points = significant_points.reshape(-1, 2)
                    rel_points = raveled_points / np.array(img_dims)
                    flat_points = rel_points.flatten()
                    yolo_format_str = "1 " + " ".join(map(str, flat_points))
                    file.write(yolo_format_str + '\n')

        print(f"Finished processing {subset} subset. Images processed: {len(list(images_dir.glob('*.png')))}")

In [ ]:
# Define source and destination directories
src_dir = Path("/home/yzhu/MSLS_YOLO/Brain MRI Dataset of Multiple Sclerosis with Consensus Manual Lesion Segmentation and Patient Meta Information")
dest_dir = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datas")

# Convert and save datasets
convert_and_save_dataset(src_dir, dest_dir)

# Training Yolo on MICCAI

In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8s-seg.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data='data.yaml', epochs=1000, imgsz=640)


New https://pypi.org/project/ultralytics/8.1.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.24 🚀 Python-3.11.8 torch-2.1.2.post301 CUDA:0 (Quadro RTX 4000, 7967MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=data.yaml, epochs=1000, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train48, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

train: Scanning /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/trai
val: Scanning /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/val/la


Plotting labels to runs/segment/train48/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/segment/train48
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


     1/1000      4.97G       2.13      2.945          6      1.376       
                 Class     Images  Instances      Box(P          R      m

                   all        672        821      0.199      0.205      0.121     0.0439      0.171      0.188     0.0948       0.03


...

   277/1000      5.01G      1.077       1.39     0.5758     0.8806       
                 Class     Images  Instances      Box(P          R      m

                   all        672        821      0.565      0.446      0.428      0.238      0.612        0.4      0.414      0.201
Stopping training early as no improvement observed in last 100 epochs. Best results observed at epoch 177, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



277 epochs completed in 3.947 hours.
Optimizer stripped from runs/segment/train48/weights/last.pt, 23.9MB
Optimizer stripped from runs/segment/train48/weights/best.pt, 23.9MB

Validating runs/segment/train48/weights/best.pt...
Ultralytics YOLOv8.1.24 🚀 Python-3.11.8 torch-2.1.2.post301 CUDA:0 (Quadro RTX 4000, 7967MiB)
YOLOv8s-seg summary (fused): 195 layers, 11780374 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      m


                   all        672        821       0.65       0.44      0.462      0.261      0.646      0.434      0.464      0.233
                lesion        672        821       0.65       0.44      0.462      0.261      0.646      0.434      0.464      0.233
Speed: 0.2ms preprocess, 4.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/segment/train48


# Testing Yolo 

## Predictions

In [2]:
import numpy as np
from pathlib import Path
from tqdm import tqdm
from ultralytics import YOLO

# Load the pretrained YOLO model
model = YOLO('best.pt')  # Make sure this points to your actual model file

# Define dataset directory
dataset_dir = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets")
output = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/data")
test_images_dir = dataset_dir / 'test' / 'images'
output_dir = output / 'test' / 'contours'
output_dir.mkdir(parents=True, exist_ok=True)

# Collect all test image paths
image_paths = [str(p) for p in test_images_dir.glob('*.png')]

# Process images
for image_path in tqdm(image_paths):
    # Perform inference on the image
    results = model.predict(image_path)
    
    # Prepare output file path for contours
    image_name = Path(image_path).stem
    output_path = output_dir / f"{image_name}.txt"

    # Check if the masks are available in the results
    if results[0].masks is not None and results[0].masks.xyn is not None:
        normalized_contours = results[0].masks.xyn  # Retrieve normalized contour points from the predictions

        # Open the output file
        with open(output_path, 'w') as f:
            for contour in normalized_contours:
                # Format contour points as "1 x1 y1 x2 y2 ... xn yn"
                formatted_points = '1 ' + ' '.join([f'{p[0]} {p[1]}' for p in contour])
                f.write(formatted_points + '\n')
    else:
        # No masks found for image, create an empty file
        with open(output_path, 'w') as f:
            pass
        print(f"Created empty file for image: {image_path}")

  0%|                                           | 0/1008 [00:00<?, ?it/s]


image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07003SATH_flair_251.png: 640x640 10 lesions, 16.0ms
Speed: 2.3ms preprocess, 16.0ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 640)


  0%|                                   | 1/1008 [00:00<03:16,  5.12it/s]


image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07040DORE_flair_317.png: 640x640 (no detections), 16.8ms
Speed: 2.4ms preprocess, 16.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07040DORE_flair_317.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07001MOEL_flair_258.png: 640x640 (no detections), 16.0ms
Speed: 1.6ms preprocess, 16.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07001MOEL_flair_258.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07003SATH_flair_101.png: 640x640 (no detections), 16.0ms
Speed: 1.4ms preprocess, 16.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv

  0%|▏                                  | 5/1008 [00:00<00:52, 19.23it/s]

...

 99%|████████████████████████████████▋| 999/1008 [00:19<00:00, 53.17it/s]

Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07001MOEL_flair_149.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07001MOEL_flair_100.png: 640x640 (no detections), 9.4ms
Speed: 1.5ms preprocess, 9.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07001MOEL_flair_100.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07040DORE_flair_105.png: 640x640 (no detections), 9.3ms
Speed: 1.4ms preprocess, 9.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07040DORE_flair_105.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07003SATH_flair_044.png: 640x640 (no detections), 9.3ms
Speed: 1.4ms preprocess, 9.3ms inference, 0.4ms postpro

100%|███████████████████████████████▉| 1005/1008 [00:19<00:00, 53.87it/s]

Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07003SATH_flair_155.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07003SATH_flair_095.png: 640x640 (no detections), 9.4ms
Speed: 1.5ms preprocess, 9.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07003SATH_flair_095.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07040DORE_flair_297.png: 640x640 (no detections), 9.4ms
Speed: 1.0ms preprocess, 9.4ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07040DORE_flair_297.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasets/test/images/07040DORE_flair_228.png: 640x640 1 lesion, 9.4ms
Speed: 1.0ms preprocess, 9.4ms inference, 1.7ms postprocess pe

100%|████████████████████████████████| 1008/1008 [00:19<00:00, 51.37it/s]


## Create mask from Yolo contours

In [3]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Set the path
dataset_dir = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/data/test")
contours_dir = dataset_dir / 'contours'  # Assume contour files are saved in this directory
output_masks_dir = dataset_dir / 'predictions'  # Save the generated masks here
output_masks_dir.mkdir(exist_ok=True)  # Make sure the output directory exists

# Original image dimensions
image_width, image_height = 261, 336  # Adjust according to the actual dimensions

for contour_file in tqdm(list(contours_dir.glob('*.txt')), desc="Processing contours"):
    # Read and parse the contour file
    with open(contour_file, 'r') as file:
        contours = []
        for line in file:
            if line.strip():  # Ignore empty lines
                parts = line.split()
                # Parse normalized coordinates and convert to image coordinates
                contour = [(float(parts[i]), float(parts[i+1])) for i in range(1, len(parts), 2)]
                contour = np.array(contour) * np.array([image_width, image_height])
                contours.append(contour.astype(int))

    # Create a blank mask image
    mask = np.zeros((image_height, image_width), dtype=np.uint8)

    # Draw contours
    for contour in contours:
        cv2.fillPoly(mask, [contour], 255)  # Use fillPoly to fill the contour

    # Save the mask image
    output_path = output_masks_dir / contour_file.name.replace('.txt', '.png')
    cv2.imwrite(str(output_path), mask)

Processing contours: 100%|█████████| 1008/1008 [00:00<00:00, 3839.75it/s]


## Build 3D from 2D masks

In [1]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import imageio.v2 as imageio
import nibabel as nib
# Define paths
dataset_dir = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/data/test")
test_images_dir = dataset_dir / 'images'
test_predictions_dir = dataset_dir / 'predictions'
test_masks_dir = dataset_dir / 'labels'

output_nii_dir = dataset_dir / 'nii'
output_nii_dir.mkdir(exist_ok=True)  # Make sure the output directory exists

# Get the IDs of all patients
patients = set(f.stem.split('_')[0] for f in test_images_dir.glob('*.png'))

# Reconstruct 3D volumes for each patient
for patient in patients:
    image_files = sorted(test_images_dir.glob(f'{patient}*.png'))
    prediction_files = [test_predictions_dir / f.name for f in image_files]  # Assuming mask and original image filenames are the same
    mask_files = [test_masks_dir / f.name.replace('flair', 'consensus') for f in image_files] # Assuming mask and original image filenames are the same

    # Read images and masks, and construct 3D arrays
    images = np.stack([imageio.imread(f) for f in tqdm(image_files, desc=f'Reading images for {patient}')], axis=-1)
    predictions = np.stack([imageio.imread(f) for f in tqdm(prediction_files, desc=f'Reading masks for {patient}')], axis=-1)
    masks = np.stack([imageio.imread(f) for f in tqdm(mask_files, desc=f'Reading masks for {patient}')], axis=-1)

    # Create NIfTI images and save them
    #images_nifti = nib.Nifti1Image(images, affine=np.eye(4))
    predictions_nifti = nib.Nifti1Image(predictions, affine=np.eye(4))
    
    masks_nifti = nib.Nifti1Image(masks, affine=np.eye(4))
    flair_image = sitk.ReadImage(flair_path)
    masks_nifti.CopyInformation(flair_image)

    nib.save(images_nifti, str(output_nii_dir / f'{patient}_images.nii.gz'))
    nib.save(predictions_nifti, str(output_nii_dir / f'{patient}_predictions.nii.gz'))
    nib.save(masks_nifti, str(output_nii_dir / f'{patient}_masks.nii.gz'))

Reading masks for 07001MOEL: 100%|██████████| 336/336 [00:00<00:00, 2029.00it/s]


In [ ]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import imageio.v2 as imageio
import nibabel as nib
# Define paths
dataset_dir = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/data/test")
test_images_dir = dataset_dir / 'images'
test_predictions_dir = dataset_dir / 'predictions'
test_masks_dir = dataset_dir / 'labels'

output_nii_dir = dataset_dir / 'nii'
output_nii_dir.mkdir(exist_ok=True)  # Make sure the output directory exists

# Get the IDs of all patients
patients = set(f.stem.split('_')[0] for f in test_images_dir.glob('*.nii'))

# Reconstruct 3D volumes for each patient
for patient in patients:
    # Load the images
    flair_image = sitk.ReadImage(flair_path)

    # Assuming that the third dimension Z is the slice direction
    z_slices = range(flair_image.GetSize()[2])
    image_width = lair_image.GetSize()[0]
    image_height = lair_image.GetSize()[1]
    
    masks_nifti = SITK.iMAGE...
    masks_nifti.CopyInformation(flair_image)
    
    # Process each slice and save to the appropriate directory
    for z_slice in z_slices:
        image = sitk.Cast(sitk.RescaleIntensity(flair_image[:, :, z_slice]), sitk.sitkUInt8)
        image_resized = cv2.resize(image, (640,640) )
        
        contours_predicted = model.predict(image_resized)

        # Create a blank mask image
        mask = np.zeros((image_height, image_width), dtype=np.uint8)

        # Draw contours
        for contour in contours_predicted:
            cv2.fillPoly(mask, [contour], 255)  # Use fillPoly to fill the contour

        masks_nifti [:, :, z_slice] = mask
    
    sitk.WtriteImage(masks_nifti,f'{patient}_masks.nii.gz' )
    

In [1]:
%pip install imageio

Note: you may need to restart the kernel to use updated packages.


In [41]:
import SimpleITK as sitk
import os
import re
from sklearn.model_selection import train_test_split

def save_image(image, output_path):
    # Ensure the image is in the correct format (unsigned char) for PNG
    image = sitk.Cast(sitk.RescaleIntensity(image), sitk.sitkUInt8)
    sitk.WriteImage(image, output_path)

def process_case(case_dir, output_dir, case_name):
    print(f'Processing case: {case_name}...')
    # Extract the last number from the case name
    last_number = re.findall(r'\d+', case_name)[-1]  # Find all numbers and take the last one

    # Paths to the image files, modified to start with the last number of the folder
    consensus_path = os.path.join(case_dir, f"{last_number}-LesionSeg-Flair.nii")
    flair_path = os.path.join(case_dir, f"{last_number}-Flair.nii")
    
    # Load the images
    consensus_image = sitk.ReadImage(consensus_path)
    flair_image = sitk.ReadImage(flair_path)

    # Assuming that the third dimension Z is the slice direction
    z_slices = range(consensus_image.GetSize()[2])
    
    # Process each slice and save to the appropriate directory
    for z_slice in z_slices:
        save_image(consensus_image[:, :, z_slice], os.path.join(output_dir, 'labels', f"{case_name}_consensus_{z_slice:03}.png"))
        save_image(flair_image[:, :, z_slice], os.path.join(output_dir, 'images', f"{case_name}_flair_{z_slice:03}.png"))

    print(f'Finished processing case: {case_name}')

# Directories for train and test datasets
output_dir = './datas'
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
valid_dir = os.path.join(output_dir, 'valid')

# Create directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(valid_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(valid_dir, 'labels'), exist_ok=True)

# Root directory of the cases
data_root_dir = '/home/yzhu/MSLS_YOLO/Brain MRI Dataset of Multiple Sclerosis with Consensus Manual Lesion Segmentation and Patient Meta Information'

# Get a list of all case directories
test_cases = [d for d in os.listdir(data_root_dir) if os.path.isdir(os.path.join(data_root_dir, d))]




for case_name in test_cases:
    case_dir = os.path.join(data_root_dir, case_name)
    process_case(case_dir, test_dir, case_name)

Processing case: Patient-17...
Finished processing case: Patient-17
Processing case: Patient-45...
Finished processing case: Patient-45
Processing case: Patient-29...
Finished processing case: Patient-29
Processing case: Patient-37...
Finished processing case: Patient-37
Processing case: Patient-22...
Finished processing case: Patient-22
Processing case: Patient-12...
Finished processing case: Patient-12
Processing case: Patient-59...
Finished processing case: Patient-59
Processing case: Patient-3...
Finished processing case: Patient-3
Processing case: Patient-42...
Finished processing case: Patient-42
Processing case: Patient-24...
Finished processing case: Patient-24
Processing case: Patient-30...
Finished processing case: Patient-30
Processing case: Patient-33...
Finished processing case: Patient-33
Processing case: Patient-54...
Finished processing case: Patient-54
Processing case: Patient-25...
Finished processing case: Patient-25
Processing case: Patient-58...
Finished processing

In [42]:
import cv2
import numpy as np
from pathlib import Path
import random
import os
from shutil import copy2

def resize_and_save_image(image_path, dest_path, scale_dims=(256,256)):
    """
    Resize an image and save it to the destination path.
    """
    image = cv2.imread(str(image_path))
    image_resized = cv2.resize(image, scale_dims)
    cv2.imwrite(str(dest_path), image_resized)

def find_significant_points(contour, angle_threshold=15):
    """
    Simplify a contour to include points with significant changes in direction.
    """
    def angle_between(p1, p2, p3):
        """
        Calculate the angle between three points.
        """
        vector1 = p2 - p1
        vector2 = p3 - p2
        unit_vector1 = vector1 / np.linalg.norm(vector1)
        unit_vector2 = vector2 / np.linalg.norm(vector2)
        dot_product = np.dot(unit_vector1, unit_vector2)
        angle = np.arccos(dot_product)
        return np.degrees(angle)

    significant_points = [contour[0]]
    for i in range(1, len(contour) - 1):
        angle = angle_between(contour[i - 1][0], contour[i][0], contour[i + 1][0])
        if angle > angle_threshold:
            significant_points.append(contour[i])
    significant_points.append(contour[-1])
    return np.array(significant_points)

def convert_and_save_dataset(src_dir, dest_dir, img_dims=(640, 640)):
    """
    Process datasets, resize images and masks, and convert masks to YOLO format.
    """
    subsets = ['train','val', 'test']
    all_images = []

    for subset in subsets:
        print(f"Processing {subset} subset...")

        images_dir = src_dir / subset / 'images'
        labels_dir = src_dir / subset / 'labels'
        dest_images_dir = dest_dir / subset / 'images'
        dest_labels_dir = dest_dir / subset / 'labels'
        
        dest_images_dir.mkdir(parents=True, exist_ok=True)
        dest_labels_dir.mkdir(parents=True, exist_ok=True)
        
        for image_path in images_dir.glob("*.png"):
            # Resize and save image
            dest_image_path = dest_images_dir / image_path.name
            resize_and_save_image(image_path, dest_image_path, img_dims)

            # Process and convert mask
            mask_name = image_path.name.replace("flair", "consensus")
            mask_path = labels_dir / mask_name
            mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
            mask_resized = cv2.resize(mask, img_dims)
            contours, _ = cv2.findContours(mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            yolo_annotation_file = dest_labels_dir / mask_name.replace('consensus', 'flair').replace('.png', '.txt')
            with open(yolo_annotation_file, 'w') as file:
                for contour in contours:
                    significant_points = find_significant_points(contour)
                    raveled_points = significant_points.reshape(-1, 2)
                    rel_points = raveled_points / np.array(img_dims)
                    flat_points = rel_points.flatten()
                    yolo_format_str = "1 " + " ".join(map(str, flat_points))
                    file.write(yolo_format_str + '\n')

        print(f"Finished processing {subset} subset. Images processed: {len(list(images_dir.glob('*.png')))}")
# Define source and destination directories
src_dir = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datas")
dest_dir = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss")

# Convert and save datasets
convert_and_save_dataset(src_dir, dest_dir)

Processing train subset...
Finished processing train subset. Images processed: 0
Processing val subset...
Finished processing val subset. Images processed: 0
Processing test subset...
Finished processing test subset. Images processed: 1451


In [43]:
import numpy as np
from pathlib import Path
from tqdm import tqdm
from ultralytics import YOLO

# Load the pretrained YOLO model
model = YOLO('best.pt')  # Make sure this points to your actual model file

# Define dataset directory
dataset_dir = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss")
output = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datas")
test_images_dir = dataset_dir / 'test' / 'images'
output_dir = output / 'test' / 'contours'
output_dir.mkdir(parents=True, exist_ok=True)

# Collect all test image paths
image_paths = [str(p) for p in test_images_dir.glob('*.png')]

# Process images
for image_path in tqdm(image_paths):
    # Perform inference on the image
    results = model.predict(image_path)
    
    # Prepare output file path for contours
    image_name = Path(image_path).stem
    output_path = output_dir / f"{image_name}.txt"

    # Check if the masks are available in the results
    if results[0].masks is not None and results[0].masks.xyn is not None:
        normalized_contours = results[0].masks.xyn  # Retrieve normalized contour points from the predictions

        # Open the output file
        with open(output_path, 'w') as f:
            for contour in normalized_contours:
                # Format contour points as "1 x1 y1 x2 y2 ... xn yn"
                formatted_points = '1 ' + ' '.join([f'{p[0]} {p[1]}' for p in contour])
                f.write(formatted_points + '\n')
    else:
        # No masks found for image, create an empty file
        with open(output_path, 'w') as f:
            pass
        print(f"Created empty file for image: {image_path}")


  0%|                                                  | 0/1451 [00:00<?, ?it/s]


image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-34_flair_004.png: 640x640 (no detections), 13.4ms
Speed: 1.1ms preprocess, 13.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)



  0%|                                          | 1/1451 [00:00<04:21,  5.55it/s]

Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-34_flair_004.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-50_flair_007.png: 640x640 (no detections), 13.5ms
Speed: 1.2ms preprocess, 13.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-50_flair_007.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-53_flair_014.png: 640x640 1 lesion, 13.5ms
Speed: 1.2ms preprocess, 13.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-18_flair_019.png: 640x640 (no detections), 13.4ms
Speed: 1.2ms preprocess, 13.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOL


  0%|▏                                         | 6/1451 [00:00<01:02, 23.05it/s]

...

Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-54_flair_004.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-22_flair_001.png: 640x640 (no detections), 8.0ms
Speed: 1.1ms preprocess, 8.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-22_flair_001.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-49_flair_011.png: 640x640 (no detections), 7.9ms
Speed: 1.1ms preprocess, 7.9ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-49_flair_011.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-34_flair_001.png: 640x640 (no detections), 7.9ms
Speed: 1.0ms preprocess, 7.9ms 


100%|██████████████████████████████████████▉| 1449/1451 [00:25<00:00, 59.14it/s]


image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-37_flair_005.png: 640x640 (no detections), 8.0ms
Speed: 1.1ms preprocess, 8.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-37_flair_005.png

image 1/1 /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-38_flair_013.png: 640x640 (no detections), 8.0ms
Speed: 1.1ms preprocess, 8.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


100%|███████████████████████████████████████| 1451/1451 [00:25<00:00, 56.58it/s]

Created empty file for image: /home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datasetssss/test/images/Patient-38_flair_013.png


In [44]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Set the path
dataset_dir = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datas/test")
contours_dir = dataset_dir / 'contours'  # Assume contour files are saved in this directory
output_masks_dir = dataset_dir / 'predictions'  # Save the generated masks here
output_masks_dir.mkdir(exist_ok=True)  # Make sure the output directory exists

# Original image dimensions
image_width, image_height = 256,256  # Adjust according to the actual dimensions

for contour_file in tqdm(list(contours_dir.glob('*.txt')), desc="Processing contours"):
    # Read and parse the contour file
    with open(contour_file, 'r') as file:
        contours = []
        for line in file:
            if line.strip():  # Ignore empty lines
                parts = line.split()
                # Parse normalized coordinates and convert to image coordinates
                contour = [(float(parts[i]), float(parts[i+1])) for i in range(1, len(parts), 2)]
                contour = np.array(contour) * np.array([image_width, image_height])
                contours.append(contour.astype(int))

    # Create a blank mask image
    mask = np.zeros((image_height, image_width), dtype=np.uint8)

    # Draw contours
    for contour in contours:
        cv2.fillPoly(mask, [contour], 255)  # Use fillPoly to fill the contour

    # Save the mask image
    output_path = output_masks_dir / contour_file.name.replace('.txt', '.png')
    cv2.imwrite(str(output_path), mask)


Processing contours: 100%|████████████████| 1451/1451 [00:00<00:00, 4094.99it/s]


In [45]:
import numpy as np
import nibabel as nib
from pathlib import Path
from tqdm import tqdm
import imageio.v2 as imageio

def read_and_stack_slices(directory, file_pattern):
    """Read PNG files from directory and stack into a 3D array."""
    files = sorted(directory.glob(file_pattern))  # Ensure files are sorted to maintain correct order
    slices = [imageio.imread(f) for f in files]
    return np.stack(slices, axis=0)  # Stack along the first dimension (no need to transpose)

def process_patient_data(patient_id, dirs):
    """Process and reconstruct 3D NIfTI files for images, predictions, and masks."""
    images_3d = read_and_stack_slices(dirs['images'], f'{patient_id}_*.png')
    predictions_3d = read_and_stack_slices(dirs['predictions'], f'{patient_id}_*.png')
    masks_3d = read_and_stack_slices(dirs['masks'], f'{patient_id}_*.png')

    # Convert numpy arrays to NIfTI images and save
    images_nii = nib.Nifti1Image(images_3d, affine=np.eye(4))
    predictions_nii = nib.Nifti1Image(predictions_3d, affine=np.eye(4))
    masks_nii = nib.Nifti1Image(masks_3d, affine=np.eye(4))
    
    nib.save(images_nii, dirs['output'] / f'{patient_id}_images.nii.gz')
    nib.save(predictions_nii, dirs['output'] / f'{patient_id}_predictions.nii.gz')
    nib.save(masks_nii, dirs['output'] / f'{patient_id}_masks.nii.gz')

# Paths and directories
base_dir = Path("/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/datas/test")
dirs = {
    'images': base_dir / 'images',
    'predictions': base_dir / 'predictions',
    'masks': base_dir / 'labels',
    'output': base_dir / 'nii'
}
dirs['output'].mkdir(exist_ok=True)

# Process each patient
patients = set(f.stem.split('_')[0] for f in dirs['images'].glob('*.png'))

# Commented out to prevent accidental execution
for patient in tqdm(patients, desc="Processing patients"):
    process_patient_data(patient, dirs)



Processing patients: 100%|██████████████████████| 60/60 [00:08<00:00,  7.00it/s]


In [50]:
import numpy as np
import nibabel as nib
from pathlib import Path
from sklearn.metrics import confusion_matrix
import pandas as pd

# Function to compute the confusion matrix and Dice score
def compute_metrics(nii_mask_path, nii_prediction_path, threshold=0.5):
    # Load the ground truth mask and the prediction
    mask_nii = nib.load(nii_mask_path)
    prediction_nii = nib.load(nii_prediction_path)
    
    # Ensure that both images have the same shape
    assert mask_nii.shape == prediction_nii.shape, "Shapes of mask and prediction must match."
    
    # Flatten the arrays to 1D for comparison
    mask_data = mask_nii.get_fdata().astype(bool).flatten()  # Convert to boolean for mask
    prediction_data = (prediction_nii.get_fdata().flatten() > threshold).astype(bool)

    # Calculate TN, TP, FN, FP
    tn, fp, fn, tp = confusion_matrix(mask_data, prediction_data, labels=[False, True]).ravel()

    # Compute precision and recall
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    # Compute Dice similarity coefficient
    dice = (2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) != 0 else 1

    return tn, tp, fn, fp, dice, f1_score

# Function to process all patients and compile metrics into a table
def process_all_patients(base_dir):
    dirs = {
        'output': base_dir / 'nii'
    }

    metrics = {
        'Patient': [],
        'TN': [],
        'TP': [],
        'FN': [],
        'FP': [],
        'Dice': [],
        'F1': []
    }

    # Loop through each patient's masks and predictions
    for mask_path in dirs['output'].glob('*_masks.nii.gz'):
        patient_id = mask_path.stem.split('_masks')[0]
        prediction_path = dirs['output'] / f'{patient_id}_predictions.nii.gz'

        # Compute metrics for each patient
        tn, tp, fn, fp, dice, f1_score = compute_metrics(mask_path, prediction_path)
        
        # Append the results to the metrics dictionary
        metrics['Patient'].append(patient_id)
        metrics['TN'].append(tn)
        metrics['TP'].append(tp)
        metrics['FN'].append(fn)
        metrics['FP'].append(fp)
        metrics['Dice'].append(dice)
        metrics['F1'].append(f1_score)
        
    # Convert to DataFrame
    df_metrics = pd.DataFrame(metrics)

    # Calculate average metrics across all patients
    avg_metrics = df_metrics.mean(numeric_only=True)
    avg_metrics['Patient'] = 'Average'

    # Append the average metrics to the DataFrame
    df_metrics = pd.concat([df_metrics, pd.DataFrame([avg_metrics])], ignore_index=True)

    return df_metrics

# Set the base directory as per the paths mentioned in the user's code
base_dir = Path('/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/data/test')  # Update this path as needed

# Run the processing function and save the DataFrame to a CSV
df_metrics = process_all_patients(base_dir)
df_metrics.to_csv(base_dir / 'patient_metrics.csv', index=False)

print(df_metrics)


     Patient            TN            TP       FN            FP      Dice  \
0  07003SATH  2.931825e+07  71716.000000  50821.0  25072.000000  0.653970   
1  07001MOEL  2.945871e+07   2260.000000   2430.0   2455.000000  0.480595   
2  07040DORE  2.944770e+07   7749.000000   4247.0   6159.000000  0.598286   
3    Average  2.940822e+07  27241.666667  19166.0  11228.666667  0.577617   

         F1  
0  0.653970  
1  0.480595  
2  0.598286  
3  0.577617  


In [5]:
import numpy as np
import nibabel as nib
from pathlib import Path
from sklearn.metrics import confusion_matrix
import pandas as pd

def compute_metrics(mask_nii, prediction_nii, threshold=0.5):
    # Flatten the arrays to 1D for comparison
    mask_data = mask_nii.get_fdata().astype(bool).flatten()  # Convert to boolean for mask
    prediction_data = (prediction_nii.get_fdata().flatten() > threshold).astype(bool)

    # Calculate TN, TP, FN, FP
    tn, fp, fn, tp = confusion_matrix(mask_data, prediction_data, labels=[False, True]).ravel()

    # Compute precision and recall
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0

    # Compute Dice similarity coefficient
    dice = (2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) != 0 else 0

    return tn, tp, fn, fp, dice, precision, recall

def process_all_patients(base_dir):
    dirs = {'output': base_dir / 'nii'}
    metrics = {'Patient': [], 'TN': [], 'TP': [], 'FN': [], 'FP': [], 'Dice': [], 'Precision': [], 'Recall': []}
    skipped_patients = []

    for mask_path in dirs['output'].glob('*_masks.nii.gz'):
        patient_id = mask_path.stem.split('_masks')[0]
        prediction_path = dirs['output'] / f'{patient_id}_predictions.nii.gz'

        mask_nii = nib.load(mask_path)
        prediction_nii = nib.load(prediction_path)

        if mask_nii.shape != prediction_nii.shape:
            print(f"Skipping patient {patient_id} due to shape mismatch.")
            skipped_patients.append(patient_id)
            continue

        tn, tp, fn, fp, dice, precision, recall = compute_metrics(mask_nii, prediction_nii)
        
        # Add to metrics only if dice is not zero to avoid affecting the mean incorrectly
        if dice != 0:
            metrics['Patient'].append(patient_id)
            metrics['TN'].append(tn)
            metrics['TP'].append(tp)
            metrics['FN'].append(fn)
            metrics['FP'].append(fp)
            metrics['Dice'].append(dice)
            metrics['Precision'].append(precision)
            metrics['Recall'].append(recall)

    df_metrics = pd.DataFrame(metrics)
    
    # If no valid data (all dice are zero), skip average calculation to avoid NaN values
    if not df_metrics.empty:
        avg_metrics = df_metrics.mean(numeric_only=True)
        avg_metrics['Patient'] = 'Average'
        df_metrics = pd.concat([df_metrics, pd.DataFrame([avg_metrics])], ignore_index=True)

    return df_metrics, skipped_patients

# Example usage (commented out to adhere to the instruction of not executing code in the chat):
# base_dir = Path('/path/to/data')
# df_metrics, skipped_patients = process_all_patients(base_dir)
# print(df_metrics)
# print(f"Skipped patients: {skipped_patients}")
# df_metrics.to_csv(base_dir / 'patient_metrics.csv', index=False)



# Then, when calling the function, you would handle the skipped patients separately:

base_dir = Path('/home/yzhu/MSLS_YOLO/UNET_YOLO_final/YOLOv8/data/test')

# Run the processing function and handle skipped patients
df_metrics, skipped_patients = process_all_patients(base_dir)

# Optionally, save the skipped patients to a file or print them
print(f"Skipped patients: {skipped_patients}")

# Save the DataFrame to a CSV
df_metrics.to_csv(base_dir / 'patient_metrics.csv', index=False)

print(df_metrics)


Skipped patients: []
     Patient            TN            TP       FN            FP      Dice  \
0  07003SATH  2.931825e+07  71716.000000  50821.0  25072.000000  0.653970   
1  07001MOEL  2.945871e+07   2260.000000   2430.0   2455.000000  0.480595   
2  07040DORE  2.944770e+07   7749.000000   4247.0   6159.000000  0.598286   
3    Average  2.940822e+07  27241.666667  19166.0  11228.666667  0.577617   

   Precision    Recall  
0   0.740960  0.585260  
1   0.479321  0.481876  
2   0.557161  0.645965  
3   0.592481  0.571034  
